## Das initiale Setup

Zunächst importieren wir die benötigten Programmbibliotheken.

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from networkx.algorithms import bipartite
from networkx.algorithms import centrality

Nun lesen wir die "Soziomatrix" ein, welche als CSV-Datei vorliegt und speichern sie im Datentyp "Dataframe" der Bibliothek pandas ab.

In [ ]:
# Assuming 'Soziomatrix_II - Tabellenblatt1.csv' is available in the current directory
df = pd.read_csv("Soziomatrix_II - Tabellenblatt1.csv")

## Die Generation des bipartiten Affiliationsnetzwerks

Wir erstellen einen ungerichteten Graphen, sowie ein Dictionary für die Individuen, eine Liste für die Gelegenheit, sich zu begegnen und eine Kantenliste. Danach speichern wir die Individuen und Gelegenheiten aus dem Dataframe in die jeweiligen Datenstrukturen und speichern sie danach in zwei Knotenklassen im Graphen ab (jeweils gekennzeichnet durch bipartite=1 oder bipartite=0). Wir gehen analog für die Kanten vor: Es besteht genau dann eine Kante zwischen Individuum und Gelegenheit, wenn das Individuum an dem jeweiligen Ort bzw. der Gelegenheit war. 

In [ ]:
G = nx.Graph()

individuals = {}
opportunities = []
edgelist = []

for i, node in enumerate(df.columns):
    if i == 0:
        continue
    opportunities.append(node)

for row in df.index:
    individuals[df.iloc[row].iloc[0]] = df.iloc[row].iloc[1:]

G.add_nodes_from(list(individuals.keys()), bipartite=0)
G.add_nodes_from(opportunities, bipartite=1)

for i in individuals.keys():
    for o, e in list(individuals[i].items()):
        if e == 1:
            edgelist.append((i, o))

G.add_edges_from(edgelist)

## Die Visualisierung des Graphens

Wir visualisieren den Graphen, indem wir zunächst das bipartite Layout generieren und es der Methode "draw" übergeben. Wir übergeben noch weitere stilistische Parameter und visualisieren das Netzwerk.

In [ ]:
pos1 = nx.bipartite_layout(G, nodes=list(individuals.keys()))

nx.draw(G, pos1, with_labels=True, font_weight='bold', node_size=700,
        node_color=['skyblue' if n in ['Cafè Wertvoll', 'VB Kletterhalle', 'Restaurant Dromedar', 'Cavete', 'Cineplex',
                                       'Basic Coffee', 'Restaurant Mon Ami', 'Waggonhalle']
                    else 'salmon' for n in G.nodes()])

plt.title('Bipartiter Graph')

## Die Projektion und die Analyse des Graphens

Wir projizieren das bimodale Netzwerk nun auf ein unimodales Netzwerk und berechnen einige Zentralitätskoeffizienten.

In [ ]:
G = nx.bipartite.projected_graph(G, list(G.nodes)[0:10:None])

print(f"degree centrality : {centrality.degree_centrality(G)}\n")
print(f"betweenness centraility : {centrality.betweenness_centrality(G)}\n")
print(f"betweenness centraility : {centrality.closeness_centrality(G)}\n")

In [ ]:
nx.draw(G, with_labels=True, font_weight='bold', node_color='skyblue')
plt.show()

Wir stellen fest, dass das Netzwerk sehr dicht und daher unübersichtlich ist. Wir "trimmen" das Netzwerk also, indem wir Kanten mit kleinem Gewicht löschen, sodass die Dichte gemindert wird: Kanten, deren Gewicht kleiner als 5 ist, werden gelöscht. Das ursprüngliche Graph wurde nicht gewichtet angelegt, daher berechnen wir die Kantenliste erneut und achten auf die Zahl der geteilten Gelegenheiten. Diese Methodik orientiert sich an der Methodik von Davis et al., die in meinem Vortrag besprochen wurde.

In [ ]:
min_shared_opps = 5

G_unimodal = nx.Graph()
edgelist_unimodal = []

G_unimodal.add_nodes_from(list(individuals.keys()), bipartite=0)

for i, indiv1 in enumerate(individuals.keys()):
    for j, indiv2 in enumerate(individuals.keys()):
        if i < j:  # Um Duplikate zu vermeiden
            shared_opps = set(individuals[indiv1].index[individuals[indiv1] == 1]) & set(
                individuals[indiv2].index[individuals[indiv2] == 1])
            if len(shared_opps) >= min_shared_opps:
                edgelist_unimodal.append((indiv1, indiv2, {'shared_opportunities': list(shared_opps)}))

G_unimodal.add_edges_from(edgelist_unimodal)



Auch dieses Netzwerk visualisieren wir (diesmal mit kamada-kawai) und berechnen bewährte Zentralitätsmaße.

In [ ]:
pos = nx.kamada_kawai_layout(G_unimodal)

nx.draw(G_unimodal, pos, with_labels=True, font_weight='bold', node_size=700)

print(f"degree centrality : {centrality.degree_centrality(G_unimodal)}\n")
print(f"betweenness centraility : {centrality.betweenness_centrality(G_unimodal)}\n")
print(f"betweenness centraility : {centrality.closeness_centrality(G_unimodal)}\n")

plt.show()